# 🏢 DE Immobilien Investment Dossier
Verwenden Sie die Zellen unten für Ihre Analyse. Drücken Sie **"Run All"** oder führen Sie die Zellen der Reihe nach mit ▶️ aus.

In [71]:
# Import der Logik aus Ihrer Python-Datei (max_re_price.py)
import max_re_price as re_tools

# Um sicherzustellen, dass Änderungen an der .py Datei sofort wirksam werden:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Analyse (Ist-Zustand)
**Kontext:** Hier prüfen wir, ob die Immobilie sich *jetzt gerade* trägt.
Viele Immobilien in A-Lagen (wie Starnberg) haben aktuell eine negative Cashflow-Rechnung, da die Kaufpreise schneller gestiegen sind als die Mieten.

In [72]:
# === EINGABE: STATUS QUO ===

KAUFPREIS = 469000
KALTMIETE_JAHR = 10800         # 900 € * 12
BEWIRTSCHAFTUNG_JAHR = 960     # nicht umlegbare Kosten

# Nebenkosten-Prozente (Bayern Standard)
MAKLER = 3.57
GRUNDERWERB = 3.50
NOTAR = 2.00

# ============================

# Automatische Berechnung der Nebenkosten
NEBENKOSTEN_EURO = re_tools.berechne_kaufnebenkosten(KAUFPREIS, MAKLER, GRUNDERWERB, NOTAR)

rendite, reinertrag, invest = re_tools.berechne_netto_mietrendite(
    KAUFPREIS, NEBENKOSTEN_EURO, KALTMIETE_JAHR, BEWIRTSCHAFTUNG_JAHR
)

print(f" Kaufpreis:          {KAUFPREIS:,.2f} €")
print(f" + Kaufnebenkosten:  {NEBENKOSTEN_EURO:,.2f} €  ({MAKLER+GRUNDERWERB+NOTAR}% für Makler, Steuer, Notar)")
print(f" = Gesamtinvest:     {invest:,.2f} € (Das müssen Sie heute bezahlen)")
print(f" ----------------------------------")
print(f" Kaltmiete Einnahme: {KALTMIETE_JAHR:,.2f} € p.a.")
print(f" - nicht umlegbar:   -{BEWIRTSCHAFTUNG_JAHR:,.2f} € (Verwaltung & Instandhaltung)")
print(f" = Reinertrag:       {reinertrag:,.2f} € (Vor Steuer)")
print(f" ----------------------------------")
print(f" NETTO-RENDITE:      {rendite:.2f} % (Reinertrag / Gesamtinvest)")
print(f" FAZIT:              {'⚠️ Ziemlich niedrig (unter Inflation)' if rendite < 2.5 else '✅ Solide Basis'}")

 Kaufpreis:          469,000.00 €
 + Kaufnebenkosten:  42,538.30 €  (9.07% für Makler, Steuer, Notar)
 = Gesamtinvest:     511,538.30 € (Das müssen Sie heute bezahlen)
 ----------------------------------
 Kaltmiete Einnahme: 10,800.00 € p.a.
 - nicht umlegbar:   -960.00 € (Verwaltung & Instandhaltung)
 = Reinertrag:       9,840.00 € (Vor Steuer)
 ----------------------------------
 NETTO-RENDITE:      1.92 % (Reinertrag / Gesamtinvest)
 FAZIT:              ⚠️ Ziemlich niedrig (unter Inflation)


### 2. Mietpotenzial (Kappungsgrenze)
**Kontext:** In angespannten Wohnungsmärkten (z.B. München, Starnberg, Berlin, Hamburg) gilt eine im Gesetz abgesenkte Kappungsgrenze.
Die Miete darf dort innerhalb von 3 Jahren um maximal **15%** steigen (statt regulär 20%).
Dies begrenzt Ihr Steigerungspotenzial massiv, selbst wenn der Mietspiegel eigentlich mehr erlauben würde.

In [73]:
# === EINGABE: MIETE ===

AKTUELLE_KALTMIETE = 900.00
ERHOEHUNG_PROZENT = 15       # Kappungsgrenze (15% oder 20%)

# ======================

neue_miete = re_tools.berechne_mietpotential(AKTUELLE_KALTMIETE, ERHOEHUNG_PROZENT)

print(f" Aktuelle Miete:     {AKTUELLE_KALTMIETE:.2f} €")
print(f" + Erhöhung ({ERHOEHUNG_PROZENT}%):  +{neue_miete - AKTUELLE_KALTMIETE:.2f} € (Max. zulässig in 3 Jahren)")
print(f" = Neue Miete:       {neue_miete:.2f} €")
print(f" ----------------------------------")
print(f" Potenzial p.a.:     {neue_miete * 12:,.2f} € Jahresmiete (Brutto)")

 Aktuelle Miete:     900.00 €
 + Erhöhung (15%):  +135.00 € (Max. zulässig in 3 Jahren)
 = Neue Miete:       1035.00 €
 ----------------------------------
 Potenzial p.a.:     12,420.00 € Jahresmiete (Brutto)


### 3. Leverage-Effekt & Anleihen-Vergleich
**Kontext:** Der Leverage-Effekt (Hebelwirkung mit Fremdkapital) funktioniert nur positiv, wenn die Immobilien-Rendite HÖHER ist als der Kreditzins.
Ist der Zins höher (z.B. 3.8% Zins vs. 2.5% Rendite), zahlen Sie für jeden geliehenen Euro drauf (Negativer Hebel).
Wir vergleichen zudem mit einer **risikofreien Bundesanleihe**. Eine Immobilie (Risiko: Leerstand, Heizung kaputt) muss eigentlich DEUTLICH mehr bringen als eine sichere Anleihe.

In [74]:
# === EINGABE: FINANZIERUNG ===
# Wir berechnen die Rendite nun basierend auf der NEUEN Miete (aus Schritt 2)
# KAUFPREIS & NEBENKOSTEN kommen aus Schritt 1
NEUE_MIETE_JAHR = neue_miete * 12
objekt_rendite, _, _ = re_tools.berechne_netto_mietrendite(
    KAUFPREIS, NEBENKOSTEN_EURO, NEUE_MIETE_JAHR, BEWIRTSCHAFTUNG_JAHR
)

KREDIT_ZINS = 3.80           # Aktueller Bauzins
EIGENKAPITAL_ANTEIL = 20.0   # Wieviel % Eigenkapital?
ANLEIHE_RENDITE = 2.80       # Vergleichszins (Bundesanleihe)

# ============================

ek_rendite = re_tools.berechne_leverage_effekt(objekt_rendite, KREDIT_ZINS, EIGENKAPITAL_ANTEIL)
anleihe_diff = re_tools.vergleich_bundesanleihe(ek_rendite, ANLEIHE_RENDITE)

# Detail-Rechnung für den User
details = re_tools.berechne_cashflow_detail(
    KAUFPREIS, NEBENKOSTEN_EURO, NEUE_MIETE_JAHR, BEWIRTSCHAFTUNG_JAHR, KREDIT_ZINS, EIGENKAPITAL_ANTEIL
)

print(f"--- DETAIL RECHNUNG (JÄHRLICH) ---")
print(f" Einnahmen:")
print(f"   Nettokaltmiete:      +{details['kaltmiete']:,.2f} €")
print(f" Ausgaben:")
print(f"   Nicht umlegbar:      -{details['bewirtschaftung']:,.2f} €")
print(f"   Zinsen an Bank:      -{details['zinsen']:,.2f} €  ({details['fremdkapital']:,.0f} € Kredit x {KREDIT_ZINS}%)")
print(f" ----------------------------------")
print(f" ERGEBNIS (Cashflow):    {details['cashflow']:,.2f} € (vor Steuern)")

if details['cashflow'] < 0:
    print(f" -> Sie zahlen jedes Jahr ca. {abs(details['cashflow']):,.0f} € drauf, um das Investment zu halten.")
else:
    print(f" -> Sie haben einen positiven Cashflow vor Steuern.")

print(f"\n EK-Rendite = {details['cashflow']:,.0f} € / {details['eigenkapital']:,.0f} € (Eigenkapital) ≈ {details['ek_rendite']:.2f}%")

print(f"\n--- VERGLEICH MIT BUNDESANLEIHE ({ANLEIHE_RENDITE}%) ---")
print(f"   Ihre EK-Rendite:      {ek_rendite:.2f} %")
print(f" - Sichere Anleihe:     -{ANLEIHE_RENDITE:.2f} %")
print(f" = Differenz:            {anleihe_diff:.2f} %-Punkte\n")

if anleihe_diff < 0:
    print(f" 📉 SCHLECHTER DEAL: Das Risiko wird nicht bezahlt (negative Risikoprämie).")
else:
    print(f" 📈 GUTER DEAL: Die Immobilie schlägt die Anleihe.")

--- DETAIL RECHNUNG (JÄHRLICH) ---
 Einnahmen:
   Nettokaltmiete:      +12,420.00 €
 Ausgaben:
   Nicht umlegbar:      -960.00 €
   Zinsen an Bank:      -15,550.76 €  (409,231 € Kredit x 3.8%)
 ----------------------------------
 ERGEBNIS (Cashflow):    -4,090.76 € (vor Steuern)
 -> Sie zahlen jedes Jahr ca. 4,091 € drauf, um das Investment zu halten.

 EK-Rendite = -4,091 € / 102,308 € (Eigenkapital) ≈ -4.00%

--- VERGLEICH MIT BUNDESANLEIHE (2.8%) ---
   Ihre EK-Rendite:      -4.00 %
 - Sichere Anleihe:     -2.80 %
 = Differenz:            -6.80 %-Punkte

 📉 SCHLECHTER DEAL: Das Risiko wird nicht bezahlt (negative Risikoprämie).


### 4. Steuern & Sanierung (Die Falle)
**Kontext 1 (AfA):** Sie dürfen nur das GEBÄUDE abschreiben, nicht den Grund & Boden. In teuren Lagen (Starnberg/Sylt/München) entfallen oft 70-80% des Preises auf den Boden. Ihre steuerliche Abschreibung ist daher minimal.

**Kontext 2 (15%-Grenze):** Vorsicht bei Sanierung in den ersten 3 Jahren! Übersteigen die Netto-Kosten 15% des *Gebäudewerts* (nicht Kaufpreis!), können Sie diese NICHT sofort absetzen. Sie müssen dann über 50 Jahre abgeschrieben werden – ein steuerliches Desaster.

In [75]:
# === EINGABE: STEUERN ===

BODENWERT_ANTEIL = 70.0      # In teuren Lagen oft 70-80%
PERS_STEUERSATZ = 42.0       # Ihr Grenzsteuersatz (%)
GEPLANTE_RENOVIERUNG = 0     # Geplante Kosten in den ersten 3 Jahren

# ========================

afa_betrag, steuer_back, gebaeude = re_tools.berechne_afa_vorteil(
    KAUFPREIS, BODENWERT_ANTEIL, PERS_STEUERSATZ
)

ueber_limit, limit_euro, rest = re_tools.check_15_prozent_grenze(
    KAUFPREIS, BODENWERT_ANTEIL, GEPLANTE_RENOVIERUNG
)

print(f" Gebäudewert (abschreibbar): {gebaeude:,.2f} €")
print(f" -> Nur {100-BODENWERT_ANTEIL}% vom Kaufpreis können steuerlich geltend gemacht werden.")
print(f"    (Da {BODENWERT_ANTEIL}% auf das Grundstück entfallen, das nicht abnutzt.)")
print(f" Jährliche AfA (2%):         {afa_betrag:,.2f} €")
print(f" = Steuerrückerstattung:     {steuer_back:,.2f} € pro Jahr (bei {PERS_STEUERSATZ}% Steuersatz)")
print(f"    (Das ist Ihr 'echter' Cashflow-Vorteil durch Steuern.)")

print(f"\n--- 15% RENOVIERUNGS-GRENZE (Die Falle) ---")
print(f" Maximales Netto-Budget:     {limit_euro:,.2f} € (für die ersten 3 Jahre)")
print(f" (Berechnet als 15% vom Gebäudewert. Alles darüber muss über 50 Jahre abgeschrieben werden!)")

if ueber_limit:
    print(f"⚠️ ACHTUNG: Sie liegen {abs(rest):,.2f} € DRÜBER! Kosten können nicht sofort abgesetzt werden.")
else:
    print(f"✅ OK. Sie haben noch {rest:,.2f} € Puffer.")

 Gebäudewert (abschreibbar): 140,700.00 €
 -> Nur 30.0% vom Kaufpreis können steuerlich geltend gemacht werden.
    (Da 70.0% auf das Grundstück entfallen, das nicht abnutzt.)
 Jährliche AfA (2%):         2,814.00 €
 = Steuerrückerstattung:     1,181.88 € pro Jahr (bei 42.0% Steuersatz)
    (Das ist Ihr 'echter' Cashflow-Vorteil durch Steuern.)

--- 15% RENOVIERUNGS-GRENZE (Die Falle) ---
 Maximales Netto-Budget:     21,105.00 € (für die ersten 3 Jahre)
 (Berechnet als 15% vom Gebäudewert. Alles darüber muss über 50 Jahre abgeschrieben werden!)
✅ OK. Sie haben noch 21,105.00 € Puffer.


### 5. Rückwärtsrechnung (Welcher Preis ist fair?)
Bestimmen Sie den maximalen Kaufpreis für Ihre Wunschrendite.

In [76]:
# === EINGABE: ZIEL ===

WUNSCH_RENDITE = 3.0
ZIEL_MIETE = 1035.00        # Miete nach Erhöhung

# =====================

kp, gesamt, nk = re_tools.berechne_maximalen_kaufpreis(
    ZIEL_MIETE, WUNSCH_RENDITE, MAKLER, GRUNDERWERB, NOTAR
)

print(f" Um {WUNSCH_RENDITE}% Rendite zu erreichen (bei Miete {ZIEL_MIETE}€), dürfen Sie maximal ausgeben:")
print(f" ----------------------------------")
print(f" Maximales Gesamtinvest:  {gesamt:,.2f} € (All-In Budget)")
print(f" - Kaufnebenkosten:      -{nk:,.2f} € (gehen weg)")
print(f" = MAXIMALER KAUFPREIS:   {kp:,.2f} € (Angebotspreis)")
print(f" ----------------------------------")

 Um 3.0% Rendite zu erreichen (bei Miete 1035.0€), dürfen Sie maximal ausgeben:
 ----------------------------------
 Maximales Gesamtinvest:  414,000.00 € (All-In Budget)
 - Kaufnebenkosten:      -34,427.25 € (gehen weg)
 = MAXIMALER KAUFPREIS:   379,572.75 € (Angebotspreis)
 ----------------------------------


### 6. Due Diligence / Weiche Faktoren (Prüfung vor Kauf)

**Kontext:** Neben den harten Finanzzahlen entscheiden oft versteckte Risiken über Top oder Flop. Prüfen Sie diese 4 Punkte ZWINGEND:

#### 1. Das „WEG-Konto“ & Protokolle
*   **Risiko Sonderumlage:** Ist die Instandhaltungsrücklage leer? Wenn die Heizung (1992) morgen ausfällt, müssen Sie u.U. sofort 10.000-20.000 € nachschießen.
*   **Prüfung:** Protokolle der letzten 3 Jahre anfordern! Wurde über Sanierungen (Dach/Fassade) oder Feuchtigkeitsschäden diskutiert? Das steht nur hier, nicht im Exposé.

#### 2. Die „Teilungserklärung“ (Das Grundgesetz)
*   **Wer zahlt was?** Beispiel Fenster: Sind sie Sondereigentum (Sie zahlen Austausch allein) oder Gemeinschaftseigentum (Alle zahlen)?
*   **Risiko:** Bei Bj. 1972 sind Fenster oft fällig. Kostenrisiko ~15.000 €.

#### 3. Der genaue Energiestatus (GEG-Risiko)
*   **Gasheizung 1992:** Ist es ein Niedertemperaturkessel (Bestandsschutz) oder Standardkessel (sofortige Austauschpflicht)?
*   **Energieausweis:** Klasse E/F/G? Drohende EU-Sanierungspflichten führen zu Preisabschlägen.

#### 4. Der „Faktor Mensch“ (Die Mieter)
*   **Eigenbedarf:** Bei Familie mit Kindern & langer Mietdauer (seit 2012) in Starnberg droht Härtefall. Eigenbedarfskündigung kann Jahre stocken.
*   **Prüfung:** Mietkonto einsehen (Rückstände?).

> **FAZIT:** Ohne Prüfung dieser Dokumente (Protokolle, Rücklagenstand, Teilungserklärung) ist der Preis eine "Black Box".